# Import Training Data, Add Labels, Check

In [2]:
#add labels, print dataframes to check
import pandas as pd
import numpy as np
train_map = pd.read_csv('train.map', header = None, delimiter=' ', names= ['labels', 'label_id'])
train_data = pd.read_csv('train.data', header = None, delimiter=' ', names=['doc_id', 'word_id', 'count'] )
train_label = pd.read_csv('train.label', header = None, delimiter=' ', names= ['label_id'])
train_label=train_label.reset_index()
train_label.columns = ['doc_id', 'label_id']
#add one to the doc ID so that there are no zeroes
train_label['doc_id']=train_label['doc_id']+1
print train_map[:10]
print train_data[:10]
train_label

                     labels  label_id
0               alt.atheism         1
1             comp.graphics         2
2   comp.os.ms-windows.misc         3
3  comp.sys.ibm.pc.hardware         4
4     comp.sys.mac.hardware         5
5            comp.windows.x         6
6              misc.forsale         7
7                 rec.autos         8
8           rec.motorcycles         9
9        rec.sport.baseball        10
   doc_id  word_id  count
0       1        1      4
1       1        2      2
2       1        3     10
3       1        4      4
4       1        5      2
5       1        6      1
6       1        7      1
7       1        8      1
8       1        9      3
9       1       10      9


,doc_id,label_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1


# Merge Training Data Tables

In [3]:
train_data_joined = pd.merge( train_label, train_data, on='doc_id', how="inner")
train_data_joined.head(5)

,doc_id,label_id,word_id,count
0,1,1,1,4
1,1,1,2,2
2,1,1,3,10
3,1,1,4,4
4,1,1,5,2


# Get Total Doc Count & Docs Per Class

In [4]:
#Get total document count by searching the unique values, this could also be done by taking max of the ID 
total_doc_count= train_data_joined['doc_id'].unique().shape
# Count number of docs for each class
docs_per_class=train_data_joined.groupby('label_id').doc_id.nunique()

# Calculate Pi for Classes

In [5]:
# Calculate Pi for Each Class, this is the number of documents in a class/total number of docs
df_docs_frac_per_class=np.log(train_data_joined.groupby('label_id').doc_id.nunique()/total_doc_count)
pi_values=df_docs_frac_per_class.reset_index()
pi_values.columns=['label_id','log_pi']
#pi_values

# Calculate Total Words Per Label

In [6]:
words_per_class=train_data_joined[['label_id','count']].groupby('label_id').sum().reset_index()
words_per_class.columns=['label_id','words_per_label']
#words_per_class

# Calculate Total Words Per Doc

In [7]:
sum_words_per_doc=train_data_joined.groupby(['label_id','word_id'])['count'].sum().reset_index()
sum_words_per_doc.head(5)

,label_id,word_id,count
0,1,1,13
1,1,2,63
2,1,3,275
3,1,4,9
4,1,5,82


# Set Up Training Data Probability Distribution (Pj)

# SACHIN: THIS IS WHERE I'M HAVING ISSUES, I NEED TO THE PANDAS DATAFRAME UNSTACKED AND THE REST SHOULD WORK 

In [58]:
#count of a specific word in the same class/total number of words in each class
#Alpha value for smoothing
a = 0.001

#Calculate probability of each word based on the class

pb_j=train_data_joined.groupby('label_id')
pb_ij=train_data_joined[['label_id','word_id','count']].groupby(['label_id','word_id'])
prob_distr =  (pb_ij['count'].sum() + 1) / (pb_j['count'].sum() + 61188) 
prob_distr.unstack()

#add one for all of the null values as (1/count+V+1)
for c in range(1,21):
    prob_distr.loc[c,:] = prob_distr.loc[c,:].fillna(a/(pb_j['count'].sum()[c] + 16689))
    
#Convert to dictionary for greater speed
prob_distr_dict = prob_distr.to_dict()

#prob_distr=prob_distr.reset_index()
x=prob_distr.unstack
x
###### I NEED X AS IT IS HERE Reindexed, But I cant save the instance Method

<bound method Series.unstack of label_id  word_id
1         1          0.000067
          2          0.000305
          3          0.001314
          4          0.000048
          5          0.000395
          6          0.000200
          7          0.000033
          8          0.000010
          9          0.000167
          10         0.000671
          11         0.000019
          12         0.019443
          13         0.000081
          14         0.000090
          15         0.000200
          16         0.002662
          17         0.001057
          18         0.000052
          19         0.000038
          20         0.000057
          21         0.000010
          22         0.000010
          23         0.012990
          24         0.000019
          25         0.001452
          26         0.000010
          27         0.006152
          28         0.000081
          29         0.034114
          30         0.012800
                       ...   
20        53946     

# Import Test Data

In [17]:
test_map = pd.read_csv('test.map', header = None, delimiter=' ', names= ['test_labels', 'test_label_id'])
test_data = pd.read_csv('test.data', header = None, delimiter=' ', names=['test_doc_id', 'test_word_id', 'test_count'] )
test_label = pd.read_csv('test.label', header = None, delimiter=' ', names= ['test_label_id'])
test_label=test_label.reset_index()
test_label.columns = ['test_doc_id', 'label_id']
print test_map.head(5)
print test_data.head(5)
test_label.head(5)

                test_labels  test_label_id
0               alt.atheism              1
1             comp.graphics              2
2   comp.os.ms-windows.misc              3
3  comp.sys.ibm.pc.hardware              4
4     comp.sys.mac.hardware              5
   test_doc_id  test_word_id  test_count
0            1             3           1
1            1            10           1
2            1            12           8
3            1            17           1
4            1            23           8


,test_doc_id,label_id
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


# Merge Test Data

In [18]:
#test_label['test_doc_id']=test_label['test_doc_id']
merged_test_data = pd.merge( test_label, test_data, on='test_doc_id', how="inner")
merged_test_data.head(5)

,test_doc_id,label_id,test_word_id,test_count
0,1,1,3,1
1,1,1,10,1
2,1,1,12,8
3,1,1,17,1
4,1,1,23,8


# Probability of Training Data

# Get Single Test Document for Testing

In [29]:
df=merged_test_data.loc[merged_test_data['test_doc_id']==1];
df.columns = df.columns.str.replace('test_word_id','word_id')
#sum_words_per_doc=df.groupby(['word_id','doc_id'])['count'].sum().reset_index()
sum_words_per_doc.head(5)
df.head(5)

,test_doc_id,label_id,word_id,test_count
0,1,1,3,1
1,1,1,10,1
2,1,1,12,8
3,1,1,17,1
4,1,1,23,8


# Setup Bayesian Algorithm: Map Probability for a Doc that Belongs to Each Class
#1) Match on Doc ID, #2 Pull Out Label ID, Probability Word Belongs to that class, and count of word occurence in document; sort by word ID so you can see the map of each word mapping to each class #3 multiply words to get prior probability #4 sum prior probability over each class (it can be added due to the log). #5 raname so that ID's line up and pie values for each class can be multiplied #6 add up the pi with the prior prob, can add because of the log values #7 Answer gives the probability that the document belongs to each class
#Log(Probability(word/class)^N,words)summed

In [161]:
single_doc_dist= pd.merge(prob_distr,df, on='word_id')
single_doc_dist= single_doc_dist[['label_id_x','word_id','log(prob)','test_count']].sort_values(['word_id','label_id_x'])
single_doc_dist ['prior_prob']= single_doc_dist['test_count']*single_doc_dist['log(prob)']
single_doc_dist=single_doc_dist.groupby(['label_id_x'])['prior_prob'].sum().round(3).reset_index()
single_doc_dist.columns=['label_id','prior_prob']
single_doc_dist = pd.merge (single_doc_dist,pi_values, on='label_id' )
single_doc_dist['probability']=single_doc_dist['prior_prob']+single_doc_dist['log_pi']
single_doc_dist


,label_id,prior_prob,log_pi,probability
0,1,-1853.773,-3.156025,-1856.929025
1,2,-1715.544,-2.965060,-1718.509060
2,3,-1654.120,-2.980672,-1657.100672
3,4,-1615.027,-2.954786,-1617.981786
4,5,-1572.423,-2.975441,-1575.398441
5,6,-1725.146,-2.946304,-1728.092304
6,7,-1694.801,-2.963340,-1697.764340
7,8,-1643.030,-2.946304,-1645.976304
8,9,-1727.398,-2.939570,-1730.337570
9,10,-1641.044,-2.942932,-1643.986932


In [123]:
single_doc_dist['probability'].idxmax()+1

5

# Setup Function to Classify One Document

In [162]:
def SingleDoc_NaiveBayesClassifier(doc):
    df=merged_test_data.loc[merged_test_data['test_doc_id']==doc];
    df.columns = df.columns.str.replace('test_word_id','word_id')
    ###### Copy below this line, above is for the function
    single_doc_dist= pd.merge(prob_distr,df, on='word_id')
    single_doc_dist= single_doc_dist[['label_id_x','word_id','log(prob)','test_count']].sort_values(['word_id','label_id_x'])
    single_doc_dist ['prior_prob']= single_doc_dist['test_count']*single_doc_dist['log(prob)']
    single_doc_dist=single_doc_dist.groupby(['label_id_x'])['prior_prob'].sum().round(3).reset_index()
    single_doc_dist.columns=['label_id','prior_prob']
    single_doc_dist = pd.merge (single_doc_dist,pi_values, on='label_id' )
    single_doc_dist['probability']=single_doc_dist['prior_prob']+single_doc_dist['log_pi']
    class_rating = single_doc_dist['probability'].idxmax()+1
    print class_rating
SingleDoc_NaiveBayesClassifier(2)

13


NoneType

# Setup New Classifier Function for All Words

In [40]:
#def SingleDoc_NaiveBayesClassifier(x):
df_all= merged_test_data.loc[merged_test_data['test_doc_id']==1];
df_all.columns = df_all.columns.str.replace('test_word_id','word_id')
all_doc_dist= pd.merge(prob_distr,df_all, on=['label_id','word_id'])
all_doc_dist= all_doc_dist[['label_id','word_id','log(prob)','test_count','test_doc_id']].sort_values(['word_id','label_id'])
all_doc_dist ['prior_prob']= all_doc_dist['test_count']*all_doc_dist['log(prob)']
all_doc_dist=all_doc_dist.groupby(['test_doc_id','label_id_x'])['prior_prob'].sum().reset_index()
all_doc_dist.columns=['test_doc_id','label_id','prior_prob']
all_doc_dist = pd.merge (all_doc_dist,pi_values, on='label_id' )
all_doc_dist['probability']=all_doc_dist['prior_prob']+all_doc_dist['log_pi']


#     classified_dataframe= all_doc_dist.sort_values('probability', ascending=False).groupby('test_doc_id', as_index=False).first()
#     classified_dataframe= classified_dataframe[['test_doc_id','label_id']]
#     return classified_dataframe
# classified_dataframe=SingleDoc_NaiveBayesClassifier(merged_test_data)
# print classified_dataframe

,label_id,word_id,count
0,1,1,0.000067
1,1,2,0.000305
2,1,3,0.001314
3,1,4,0.000048
4,1,5,0.000395
5,1,6,0.000200
6,1,7,0.000033
7,1,8,0.000010
8,1,9,0.000167
9,1,10,0.000671


In [32]:
prob_distr

,label_id,word_id,count
0,1,1,0.000067
1,1,2,0.000305
2,1,3,0.001314
3,1,4,0.000048
4,1,5,0.000395
5,1,6,0.000200
6,1,7,0.000033
7,1,8,0.000010
8,1,9,0.000167
9,1,10,0.000671


# Test Accuracy of Classifier

In [246]:
test_label.columns = ['doc_id', 'test_label_id']
classified_dataframe.columns = ['doc_id', 'predicted_label_id']
guess_to_real= pd.merge(classified_dataframe,test_label, on='doc_id')
guess_to_real= pd.merge()

In [247]:
guess_to_real

,doc_id,predicted_label_id,test_label_id
0,1,5,1
1,2,13,1
2,3,5,1
3,4,3,1
4,5,5,1
5,6,5,1
6,7,11,1
7,8,7,1
8,9,6,1
9,10,5,1
